In [1]:
shape_cat="mcal"
vol="vlim"
z_range="all_z"

lambda_range=0
with_r=True

In [2]:
from astropy.io import fits
from astropy.table import Table
import pandas as pd
import fitsio as fio
import numpy as np
import ast
import gc

import astropy.cosmology
from astropy.coordinates import SkyCoord  # High-level coordinates
from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
from astropy.coordinates import Angle, Latitude, Longitude  # Angles
import astropy.units as u
from IPython.display import display, Math
from tqdm.notebook import tqdm
tqdm.pandas()

/home/cz136/anaconda3/envs/balrog/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [5]:
#This header file contains all the common packages
%matplotlib inline
%run -i /home/cz136/project/sa/lib/src/header.py $shape_cat $vol $z_range 
%run -i /home/cz136/project/sa/lib/src/func.py  
%run -i /home/cz136/project/sa/lib/src/corr_func.py

members= pd.read_pickle("/home/cz136/project/sa/data/BPZ.pkl") #load BPZ catalog
red_members=pd.read_pickle("/home/cz136/project/sa/data/shape_BPZ.pkl")
red_members_index=red_members.index
red_members=None

uber=pd.read_pickle('/home/cz136/project/sa/data/{}/{}/shapes{}.pkl'.format("mcal","uber","")) 

Band                All                                                       \
Data       MEM_MATCH_ID         Z          RA        DEC         R         P   
ID                                                                             
3007825827          620  0.619567  317.699737   0.375196  0.820374  0.640158   
3007931336         4781  0.408113  319.101131   1.144699  0.877849  0.648883   
3007938592         4781  0.408113  319.136447   1.178424  0.678156  0.770375   
3007938814         4781  0.408113  319.111381   1.182301  0.409886  0.890826   
3007938890         4781  0.408113  319.081995   1.182038  0.420349  0.752846   
...                 ...       ...         ...        ...       ...       ...   
3159944414         6369  0.385452  307.502187 -49.620158  0.424342  0.829839   
3159944537         6369  0.385452  307.500625 -49.601986  0.186968  0.933188   
3159945177         6369  0.385452  307.481761 -49.600664  0.191039  0.913877   
3159945721         6369  0.385452  307.459290 -49.611175  0.421528  0.816355   
3159946033         6369  0.385452  307.528668 -49.615762  0.467721  0.836948   

Band                                             ...                         \
Data       PFREE   THETA_I   THETA_R REFMAG_FIT  ...   Theta_sat      alpha   
ID                                               ...                          
3007825827   1.0  0.913381  0.976695  21.820244  ...  -83.935727  30.771164   
3007931336   1.0  1.000000  0.180206  18.738276  ...   93.480353 -36.968744   
3007938592   1.0  1.000000  0.998962  19.626484  ...  142.337773  80.120873   
3007938814   1.0  1.000000  1.000000  20.682962  ...  118.191081 -41.008536   
3007938890   1.0  1.000000  1.000000  18.450119  ...   60.227921  17.411425   
...          ...       ...       ...        ...  ...         ...        ...   
3159944414   1.0  1.000000  1.000000  18.103125  ...   99.667584 -63.427485   
3159944537   1.0  0.621109  1.000000  20.785936  ...  108.035780   2.284136   
3159945177   1.0  1.000000  1.000000  20.572025  ...   57.038532 -86.733879   
3159945721   1.0  1.000000  1.000000  18.292593  ...   45.271952 -44.672993   
3159946033   1.0  1.000000  1.000000  20.097336  ...  129.636920  71.498087   

Band                               \
Data              phi        angR   
ID                                  
3007825827  65.293109  979.552924   
3007931336  49.550903  785.265738   
3007938592  62.216899  785.265738   
3007938814  20.800383  785.265738   
3007938890  42.816496  785.265738   
...               ...         ...   
3159944414  16.904931  758.304353   
3159944537  74.248356  758.304353   
3159945177  36.227589  758.304353   
3159945721  89.944945  758.304353   
3159946033  58.138833  758.304353   

Band                                                                     \
Data                                           preprocessed_e         e   
ID                                                                        
3007825827  (0.3123682859357879, 0.11154588520739155, 0.29...  0.312368   
3007931336  (0.06993514090919284, 0.009872371661777879, 0....  0.069935   
3007938592  (0.4224477342035747, -0.4218489959472032, -0.0...  0.422448   
3007938814  (0.08225708000062744, -0.05961940250728352, 0....  0.082257   
3007938890  (0.1368085249907695, -0.06825620488477381, 0.1...  0.136809   
...                                                       ...       ...   
3159944414  (0.03645409870412611, -0.026731194359617506, -...  0.036454   
3159944537  (0.19171446869920045, -0.12769678992626002, 0....  0.191714   
3159945177  (0.16193247589014942, -0.15875119421838596, 0....  0.161932   
3159945721  (0.15551734137566006, -0.07881604319595942, 0....  0.155517   
3159946033  (0.2017078593946568, 0.04968903807839871, 0.19...  0.201708   

Band                                                   
Data              ep        ex       normLR  R_LAMBDA  
ID                                                     
3007825827  0.111546  0.291773  10

# foreground background

In [ ]:
ng_matrix_fore=[[None for i in range(4)] for j in range(3)]

for i in trange(3):
    for j in trange(4):
        ng=get_ng_source(clusters,members,z_bins[i],z_bins[i+1],lambda_bins[j],lambda_bins[j+1],foreback="fore")
        ng_values=np.copy(np.array([ng.meanlogr,ng.xi,ng.varxi]))
        ng_matrix_fore[i][j]=ng_values
        del ng
        gc.collect()

In [ ]:
fig, axs = plt.subplots(3, 4,figsize=(20,15))
for i in range(3):
    for j in range(4):
        values=ng_matrix_fore[i][j]
        print((i,j),values[1])
        axs[i,j].plot(np.exp(values[0]),values[1])
        axs[i,j].set_xscale('log')
        axs[i,j].set_yscale('log')

In [ ]:
ng_matrix_back=[[None for i in range(4)] for j in range(3)]

for i in trange(3):
    for j in trange(4):
        ng=get_ng_source(clusters,members,z_bins[i],z_bins[i+1],lambda_bins[j],lambda_bins[j+1],foreback="")
        ng_values=np.copy(np.array([ng.meanlogr,ng.xi,ng.varxi]))
        ng_matrix_back[i][j]=ng_values
        del ng
        gc.collect()

In [ ]:
fig, axs = plt.subplots(3, 4,figsize=(20,15))
for i in range(3):
    for j in range(4):
        values=ng_matrix_fore[i][j]
        print((i,j),values[0])
        axs[i,j].plot(np.exp(values[0]),values[1])
        axs[i,j].set_xscale('log')
        axs[i,j].set_yscale('log')